In [1]:
import os
import urllib
import requests
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
path= "C:\\Users\\yitzh\\Downloads\\"
data= pd.read_csv(path+"destinations_LP_crawler_Ex5.csv",encoding='ISO-8859-1')
data

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona
...,...,...
71,Venice,Venice
72,Vienna,Vienna
73,Warsaw,Warsaw
74,Zakynthos,Zakynthos


In [ ]:
new_list=[]
for city in data["city"]: 
    address= city
    api_key='AIzaS*****'
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error-response.status_code")
        else:    
            try:
                response_data=response.json()
                country=response_data['results'][0]['formatted_address'].split(',')[-1].strip()
                try:
#sometimes number appear at the end of formatted_address, if so, take earler word
                    int(country)
                    country=response_data['results'][0]['formatted_address'].split(',')[-2].strip()
                except:
                    country=country

                new_list.append(country)
            except: 
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")


new_list


In [ ]:
data['country']=new_list
data

In [ ]:

desc= list()
http= 'https://www.lonelyplanet.com/'
for i in range(len(data['city'])):
    url=http+data['country'][i]+'/'+data['city_LP'][i]
    response = requests.get(url)
    
    if response.status_code == 200:
        results_page = BeautifulSoup(response.content, 'html.parser')
        description= results_page.find('div',class_="readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg")
        desc.append(description.get_text())
    else:
        print("Error",url)
        desc.append('Error')

In [ ]:
data['Description']=desc

In [ ]:
del data['city_LP']
data.to_csv('LP_destinations.csv')